# 作成したモデルのdeployとserving

01_image_classificationで作ったモデルを、Vertex AIを使ってAPIとして利用できるようにしていきます。

Vertex AIでは、作成済みのモデルをuploadし、作成したendpointにそのモデルをdeployすることで、推論APIとして呼び出すことができるようになります。  
その手順を確認していきましょう。

In [ ]:
%%bash

USER=<<username>> # 自分の名前
BUCKET=mixi-ml-handson-2022
VERSION=001
REGION=asia-northeast1

gcloud ai models upload \
  --region=${REGION} \
  --display-name=mfashion-${USER} \
  --container-image-uri=asia-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest \
  --artifact-uri=gs://${BUCKET}/${USER}/${VERSION}

gcloud ai models list \
  --region=${REGION} \
  --filter=mfashion-${USER}
    
gcloud ai endpoints create \
  --region=${REGION} \
  --display-name=mfashion-${USER}

実行が終わったら、出力された`MODEL_ID`と、endpointを作成した際に出力された`ENDPOINT_ID` `'・・・/asia-northeast1/endpoints/<ENDPOINT_ID>'`を使って、endpointにmodelをdeployします。

In [ ]:
%%bash

USER=<<username>> # 自分の名前
REGION=asia-northeast1
MODEL_ID=<<model_id>> # 前のセルで出力されたMODEL_ID
ENDPOINT_ID=<<endpoint_id>> # 前のセルで出力されたENDPOINT_ID

gcloud ai endpoints deploy-model ${ENDPOINT_ID} \
  --region=${REGION} \
  --model=${MODEL_ID} \
  --display-name=mfashion-${USER} \
  --traffic-split=0=100

上記コードの実行には数分の時間がかかります。

deployが完了したら、このendpointにリクエストを投げてみましょう。

In [ ]:
from google.cloud import aiplatform

def predict_json(project, endpoint, instances):    
    region = "asia-northeast1"
    aiplatform.init(project=project, location=region)
    endpoint = aiplatform.Endpoint(endpoint)
    response = endpoint.predict(instances=instances)
    return response.predictions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show(n_cols, n_rows, train_orig):
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(16,4))
    for ax, pixels in zip(axs.flat, train_orig):
        ax.imshow(pixels, cmap="gray")
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
import numpy as np

project_name = 'hr-mixi'
endpoint_id = <<endpoint_id>> # 最初のセルで出力されたENDPOINT_ID
labels = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot',
]

(X_train_orig, y_train_orig), (X_test_orig, y_test_orig) = fashion_mnist.load_data()
requests = X_test_orig[0:8]

## <todo> requestsをresposeが返せる形に変換してください。
## ヒント: 学習時と同じデータ形式にするために、01で行った前処理と同じことをする必要があります。
requests = ___ # expandする
requests = ___　# 正規化する
requests = requests.tolist()

predictions = predict_json(project_name, endpoint_id, ___)

for i, p in enumerate(predictions):
    print("Label: '{}' ".format(labels[np.argmax(p)]), "Pred: {}".format(p))
show(8, 1, X_test_orig[0:8])